In [1]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bla\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [81]:
import pandas as pd
from newsapi import NewsApiClient

# Init
newsapi = NewsApiClient(api_key='enter you api key here')



# /v2/everything
all_articles = newsapi.get_everything(q='Brazil',
                                      sources='bbc-news,the-verge',
                                      domains='bbc.co.uk,techcrunch.com',
                                      from_param='2020-08-26',
                                      to='2020-08-28',
                                      language='en',
        
                                      sort_by='relevancy',
                                      page=1)



df1 = pd.DataFrame.from_dict(all_articles['articles'])

In [82]:
df1

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': 'bbc-news', 'name': 'BBC News'}",None,Messi contacted Guardiola before Barca transfe...,Lionel Messi tipped off Pep Guardiola over tra...,https://www.bbc.co.uk/sport/53937544,https://ichef.bbci.co.uk/live-experience/cps/6...,2020-08-27T22:24:02Z,"Argentina forward Lionel Messi, 33, contacted ..."
1,"{'id': 'bbc-news', 'name': 'BBC News'}",None,Could Berlusconi take Italian side Monza to Se...,"Silvio Berlusconi has brought money, links to ...",https://www.bbc.co.uk/sport/football/53877763,https://ichef.bbci.co.uk/onesport/cps/624/cpsp...,2020-08-27T06:08:24Z,Former Italian Prime Minister Silvio Berluscon...
2,"{'id': 'bbc-news', 'name': 'BBC News'}",https://www.facebook.com/bbcnews,Carnival and coronavouchers: Brazil’s economic...,The disruption caused by coronavirus means tha...,https://www.bbc.co.uk/news/world-latin-america...,https://ichef.bbci.co.uk/news/1024/branded_new...,2020-08-26T00:38:37Z,Image copyrightGetty ImagesImage caption\r\n T...
3,"{'id': 'bbc-news', 'name': 'BBC News'}",https://www.facebook.com/bbcnews,In pictures: Mexico school classes resume on TV,With many families having no access to the int...,https://www.bbc.co.uk/news/world-latin-america...,https://ichef.bbci.co.uk/news/1024/branded_new...,2020-08-26T11:16:14Z,Image copyrightReuters\r\nSchool lessons have ...
4,"{'id': 'bbc-news', 'name': 'BBC News'}",None,Thiago Silva: Chelsea sign Brazil defender on ...,Brazil centre-back Thiago Silva signs for Chel...,https://www.bbc.co.uk/sport/53920170,https://ichef.bbci.co.uk/live-experience/cps/6...,2020-08-28T10:46:33Z,Brazil centre-back Thiago Silva has signed for...
5,"{'id': 'bbc-news', 'name': 'BBC News'}",None,Meet 'the biggest Sunday League team in the wo...,"Thousands of fans, selling shirts and dominati...",https://www.bbc.co.uk/sport/football/53892512,https://ichef.bbci.co.uk/onesport/cps/624/cpsp...,2020-08-26T05:25:30Z,Media playback is not supported on this device...
6,"{'id': 'bbc-news', 'name': 'BBC News'}",None,Arsenal: Mikel Arteta 'feels' Pierre-Emerick A...,"Arsenal boss Mikel Arteta is ""pretty confident...",https://www.bbc.co.uk/sport/football/53936907,https://ichef.bbci.co.uk/onesport/cps/624/cpsp...,2020-08-27T15:22:09Z,Aubameyang has scored 54 Premier League goals ...


In [83]:
from nltk.tokenize import sent_tokenize
sentences = []
for s in df1['description']:
  sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x] # flatten list

In [84]:
# Extract word vectors
word_embeddings = {}
f = open(r'data\glove.6b\glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [85]:
len(word_embeddings)

400000

In [86]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [87]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bla\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [88]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [89]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [90]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [91]:
# Extract word vectors
word_embeddings = {}
f = open('data\glove.6b\glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [92]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [93]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [94]:
from sklearn.metrics.pairwise import cosine_similarity

In [95]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [96]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [97]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [98]:
# Extract top 10 sentences as the summary
for i in range(3):
  print(ranked_sentences[i][1])

Brazil centre-back Thiago Silva signs for Chelsea on a one-year deal, with the option of another.
Silvio Berlusconi has brought money, links to big-name players and success to AC Monza - but could they go all the way to Serie A?
Lionel Messi tipped off Pep Guardiola over transfer request, Real put price tag on Serge Reguilon, Chelsea set to complete Kai Havertz deal, plus more.
